Quickstart
===========

To get started compressing opacities, we'll need to read in an opacity file. For the purposes of this tutorial,
we'll just use a small sample file.

First, let's import our package.

## Setting up the objects

In [1]:
import numpy as np

import cortecs
from cortecs.opac.opac import *
from cortecs.fit.fit import *
from cortecs.fit.fit_pca import *
from cortecs.eval.eval import *

Next, let's define our filenames. The loader for `PLATON`-style opacity files is the only one that requires separate temperature, pressure, and wavelength files; no other file type requires the `load_kwargs` argument.

In [2]:
T_filename = "temperatures.npy"
P_filename = "pressures.npy"
wl_filename = "wavelengths.npy"

cross_sec_filename = "absorb_coeffs_C2H4.npy"

load_kwargs = {
    "T_filename": T_filename,
    "P_filename": P_filename,
    "wl_filename": wl_filename,
}

With our file names defined, we can instantiate an `Opac` object as below.

In [3]:
opac_obj = Opac(cross_sec_filename, loader="platon", load_kwargs=load_kwargs)

opac_obj

/Users/arjunsavel/Desktop/research/opac_compress/src/cortecs/opac/io.py:156: RuntimeWarning: divide by zero encountered in log10
  cross_section = np.log10(cross_section)


It's normal to have a warning here. For `PLATON` opacities, very low values are set as 0. We fit the log10 opacities instead of the opacity values themselves, because there's less dynamic range on a log scale—which makes things easier to fit. Taking the log of produces a `RuntimeWarning` and returns `-np.inf`. But no need to worry—we catch these and set those infinities to a large negative number. 

Now we have an `Opac` object instantiated. The object stores information on the opacity file's fields. Let's investigate a few.

In [4]:
opac_obj.wl

array([3.00000000e-07, 3.00299510e-07, 3.00599320e-07, ...,
       2.99401875e-05, 2.99700788e-05, 3.00000000e-05])

In [5]:
opac_obj.T

array([ 100.,  200.,  300.,  400.,  500.,  600.,  700.,  800.,  900.,
       1000., 1100., 1200., 1300., 1400., 1500., 1600., 1700., 1800.,
       1900., 2000., 2100., 2200., 2300., 2400., 2500., 2600., 2700.,
       2800., 2900., 3000.])

In [6]:
opac_obj.P

array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04, 1.e+05, 1.e+06, 1.e+07, 1.e+08])

The wavelengths, temperatures, and pressures on which these opacities were evaluated are attributes of the `Opac` object.

## Fitting the opacity

Now, let's do something interesting with the `Opac` object: compress it using PCA. This works by first finding the vectors that best describe the shape of the temperature--pressure dependence of the opacity function at a *single wavelength*. Then, the code fits these eigenvectors to the temperature--pressure dependence of every other wavelength.

To do so, we'll instantiate the `Fitter` object.

In [7]:
fitter = Fitter(opac_obj, wav_ind=0, nc=3)
fitter

With our `Fitter` object set up, we can fit this serially. 

Running this code is as easy as...

In [8]:
fitter.fit()

ValueError: all values are the same at this wavelength index! Try a different one.

Oops! All of the opacity values at the first wavelength are constant. So, the algorithm can't learn the temperature--pressure dependence of the opacity. Let's try again, starting with a different wavelength index.

In [9]:
fitter = Fitter(opac_obj, wav_ind=-2, nc=3)
fitter.fit()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4616/4616 [00:00<00:00, 27418.22it/s]


This process should be pretty quick — less than a second, hopefully. 

What do we do next? Now that we've *fit* the opacity, we can *evaluate* it. Hopefully things line up well enough.

We define our last object: an `Evaluator`. This step should look pretty similar as before.

In [10]:
evaluator = Evaluator(opac_obj, fitter)

Finally, we can evaluate our algorithm on a single point in the temperature--pressure--wavelength grid.

In [11]:
temperature = 300.0
pressure = 100.0
wavelength = 2.99401875e-05

evaluator.eval(temperature, pressure, wavelength)

Array(3.9590842e-10, dtype=float32)

## Checking accuracy and speed

Saving memory isn't all that useful if we're inaccurate and take too much time. Let's check whether that's the case.

First of all, time. Let's use a lower-level routine that's a bit more apples-to-apples comparison with array-indexing.

In [12]:
temperature_ind = np.where(np.isclose(opac_obj.T, temperature))[0][0]
pressure_ind = np.where(np.isclose(opac_obj.P, pressure))[0][0]
wavelength_ind = np.where(np.isclose(opac_obj.wl, wavelength))[0][0]
pca_vectors, pca_coeffs_all_wl = evaluator.fitter_results
pca_coeffs = pca_coeffs_all_wl[wavelength_ind, :, :]

In [13]:
%%timeit
eval_pca_ind_wav(temperature_ind, pressure_ind, pca_vectors, pca_coeffs)

6.77 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Let's compare this to how long it takes to access an array.

In [14]:
%%timeit
opac_obj.cross_section[temperature_ind][pressure_ind][wavelength_ind]

228 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Accessing the array directly is clearly faster. As long as this step does not bottleneck your code, however, you should be fine.

Now, let's check the accuracy. We'll compare the log abundances, because this is the quantity to which we are sensitive in exoplanet spectroscopy.

In [15]:
AMU = 1.6605390666e-24  # atomic mass unit in cgs. From astropy!

In [16]:
array_res = opac_obj.cross_section[temperature_ind][pressure_ind][wavelength_ind]

In [17]:
eval_res = np.log10(
    evaluator.eval(temperature, pressure, wavelength)
    * evaluator.load_obj.species_weight
    * AMU
    * 1e-4
)

In [18]:
percent_error = 100 * (eval_res - array_res) / array_res
percent_error

8.942134468836937

We're reaching a 9% error in the log abundance. That might be good enough for your applications, or it might not. We recommend tuning your algorithm to the level of agreement necessary for, e.g., generating a transmission spectrum, or running an emission spectrum retrieval.